This is my test file

In [66]:

import numpy as np
import pandas as pd
import geopandas as gpd
import plotly.figure_factory as ff


from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import silhouette_score
from sklearn.datasets import load_iris, load_wine
from sklearn.cluster import DBSCAN

random_state = 11

from bokeh.models import (
    ColumnDataSource,
    HoverTool,
    LogColorMapper,
    Range1d, CustomJS, Slider
)
from bokeh.palettes import Inferno256 as palette
from bokeh.plotting import figure
from bokeh.models import ColorMapper

from bokeh.layouts import row, widgetbox

In [38]:
df = pd.read_csv('census.csv') # data from the collection and cleaning tab

In [39]:
df[df['county'].str.contains('Richmond')] 

,Unnamed: 0,id,county,state,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Estimate!!RACE!!Total population!!One race,Estimate!!RACE!!Total population!!Two or more races,Estimate!!RACE!!Total population!!One race.1,Estimate!!RACE!!Total population!!One race!!White,...,Estimate!!SEX AND AGE!!Total population!!21 years and over,Estimate!!SEX AND AGE!!Total population!!62 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over.1,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!65 years and over.1,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female
507,507,0500000US13245,Richmond County,Georgia,70.9,201463,196804,4659,196804,76073,...,144985,34309,26601,154550,73613,80937,91.0,26601,11039,15562
1870,1870,0500000US36085,Richmond County,New York,74.8,474101,462311,11790,462311,352259,...,353274,92049,73325,369598,175847,193751,90.8,73325,31369,41956
1966,1966,0500000US37153,Richmond County,North Carolina,77.3,45189,44439,750,44439,28209,...,32771,9562,7692,34731,16751,17980,93.2,7692,3353,4339
2895,2895,0500000US51159,Richmond County,Virginia,64.8,8878,8461,417,8461,5745,...,7193,2213,1846,7381,4241,3140,135.1,1846,726,1120
2944,2944,0500000US51760,Richmond city,Virginia,70.3,223787,216385,7402,216385,101498,...,170520,35516,27831,183783,85989,97794,87.9,27831,11489,16342


There were some issues dealing with the independent cities in Virginia. These are cities that are their own county-equivalent entities. Additonally,many of the cities share names with counties, so we wanted to confirm that we have it all correct.

In [40]:
df['state'].replace({
    ' Alabama': 'AL',
    ' Alaska': 'AK',
    ' American Samoa': 'AS',
    ' Arizona': 'AZ',
    ' Arkansas': 'AR',
    ' California': 'CA',
    ' Colorado': 'CO',
    ' Connecticut': 'CT',
    ' Delaware': 'DE',
    ' District of Columbia': 'DC',
    ' Florida': 'FL',
    ' Georgia': 'GA',
    ' Guam': 'GU',
    ' Hawaii': 'HI',
    ' Idaho': 'ID',
    ' Illinois': 'IL',
    ' Indiana': 'IN',
    ' Iowa': 'IA',
    ' Kansas': 'KS',
    ' Kentucky': 'KY',
    ' Louisiana': 'LA',
    ' Maine': 'ME',
    ' Maryland': 'MD',
    ' Massachusetts': 'MA',
    ' Michigan': 'MI',
    ' Minnesota': 'MN',
    ' Mississippi': 'MS',
    ' Missouri': 'MO',
    ' Montana': 'MT',
    ' Nebraska': 'NE',
    ' Nevada': 'NV',
    ' New Hampshire': 'NH',
    ' New Jersey': 'NJ',
    ' New Mexico': 'NM',
    ' New York': 'NY',
    ' North Carolina': 'NC',
    ' North Dakota': 'ND',
    ' Northern Mariana Islands':'MP',
    ' Ohio': 'OH',
    ' Oklahoma': 'OK',
    ' Oregon': 'OR',
    ' Pennsylvania': 'PA',
    ' Puerto Rico': 'PR',
    ' Rhode Island': 'RI',
    ' South Carolina': 'SC',
    ' South Dakota': 'SD',
    ' Tennessee': 'TN',
    ' Texas': 'TX',
    ' Utah': 'UT',
    ' Vermont': 'VT',
    ' Virginia': 'VA',
    ' Washington': 'WA',
    ' West Virginia': 'WV',
    ' Wisconsin': 'WI',
    ' Wyoming': 'WY'} , inplace = True) 

The cell above converts everything from spelled out names to postal abreviations. The decision was made early on to ensure that all of our data refered to states the same way to avoid conflict.

In [67]:
df.rename( columns = {'id': "fips"}, inplace = True) # line up the IDs

In [42]:
df.head()

,Unnamed: 0,fips,county,state,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Sex ratio (males per 100 females),Estimate!!RACE!!Total population,Estimate!!RACE!!Total population!!One race,Estimate!!RACE!!Total population!!Two or more races,Estimate!!RACE!!Total population!!One race.1,Estimate!!RACE!!Total population!!One race!!White,...,Estimate!!SEX AND AGE!!Total population!!21 years and over,Estimate!!SEX AND AGE!!Total population!!62 years and over,Estimate!!SEX AND AGE!!Total population!!65 years and over,Estimate!!SEX AND AGE!!Total population!!18 years and over.1,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Female,Estimate!!SEX AND AGE!!Total population!!18 years and over!!Sex ratio (males per 100 females),Estimate!!SEX AND AGE!!Total population!!65 years and over.1,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Male,Estimate!!SEX AND AGE!!Total population!!65 years and over!!Female
0,0,0500000US01001,Autauga County,AL,76.5,55200,54170,1030,54170,42437,...,39614,9403,8050,41831,20054,21777,92.1,8050,3488,4562
1,1,0500000US01003,Baldwin County,AL,86.6,208107,204535,3572,204535,179526,...,155502,49811,40665,162430,78104,84326,92.6,40665,18870,21795
2,2,0500000US01005,Barbour County,AL,76.2,25782,25429,353,25429,12216,...,19394,5658,4634,20346,10923,9423,115.9,4634,2004,2630
3,3,0500000US01007,Bibb County,AL,80.5,22527,22340,187,22340,17268,...,16864,4323,3661,17868,9578,8290,115.5,3661,1633,2028
4,4,0500000US01009,Blount County,AL,79.7,57645,56710,935,56710,55054,...,42232,12400,10233,44177,21595,22582,95.6,10233,4540,5693


In [43]:
fip = df['fips'].str.split('US',expand = True)[1]

In [44]:
fip

0       01001
1       01003
2       01005
3       01007
4       01009
        ...  
3215    72145
3216    72147
3217    72149
3218    72151
3219    72153
Name: 1, Length: 3220, dtype: object

The FIP should be a 5 digit number, so we need to make sure that the states that need a leading zero have it

In [45]:
df['fip'] = fip

In [46]:
df.drop(columns = ['Unnamed: 0'], inplace = True)

In [47]:
cols = list(df)

In [48]:
cols.insert(0,cols.pop(cols.index('fip')))

In [49]:
df = df.loc[:,cols]

In [50]:
df.drop(columns = 'fips', inplace = True)

In [68]:
# We are limiting to the Contiguous US, so we drop AK, HI, and PR
df.drop(df.loc[df['state'] == 'AK'].index, inplace=True)
df.drop(df.loc[df['state'] == 'HI'].index, inplace=True)
df.drop(df.loc[df['state'] == 'PR'].index, inplace=True)


In [69]:
df.shape

(3108, 92)

In [53]:
contig = gpd.read_file('cb_2018_us_county_500k.shp') # This is the shape file to draw the maps
values = range(len(fip))

In [54]:
contiguous_usa = gpd.read_file('cb_2018_us_county_500k.shp') 
contiguous_usa[contiguous_usa['NAME'] == 'New York']
geo = contiguous_usa

In [55]:
geo.shape

(3233, 10)

In [56]:
geo = geo[(geo['STATEFP'] != '15') & (geo['STATEFP'] != '02') & (geo['STATEFP'] != '72') & 
           (geo['STATEFP'] != '66') & (geo['STATEFP'] != '78') & (geo['STATEFP'] != '69') &
           (geo['STATEFP'] != '60')]

The origional shape file contains states and territories that we do not want to include. We will drop them from the data (and hence, the map) by their number. 

We are dropping:   
  
- Hawaii
- Alaska
- Northern Marianas
- Guam
- Puerto Rico
- American Samoa
- US Virgin Islands

In [57]:
master = pd.read_csv('../../../area_included.csv') # area_included.csv - comparison clusters of counties
growth = pd.read_csv('../../../growth .csv')    # growth.csv - week over week infection growth

In [58]:
# Switch which one is commented out based on if you want clusters (top) or growth (bottom)

# X = master.drop(columns = ['total_covid', 'deaths', 'county_x', 'county_y', 'state', 'id', 'population', 
#                            'beds_per_case', 'unnamed: 0_y', 'st_num',
#                            'cases', 'countyfips', 'county'])

X = growth['per_growth']


In [59]:
# ss = StandardScaler()

# X_scaled = ss.fit_transform(X)

# dbscan = DBSCAN(eps = 1, min_samples= 5)
# dbscan.fit(X_scaled)

In [60]:
geo['per_growth'] = X # adds growth rate to the DF for color definition in rendering

In [61]:
# geo['cluster'].value_counts()

In [64]:
import json
from bokeh.io import show
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure

geosource = GeoJSONDataSource(geojson = geo.to_json())

In [65]:
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 100) # sets color range
# Create figure object.
p = figure(title = 'US Counties', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = "pan, wheel_zoom, box_zoom, reset")
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None


counties = p.patches('xs', 'ys', source = geosource,
                    fill_color = {'field' : 'per_growth',       # sets which data will inform colors
                                 'transform' : color_mapper},
                    line_color = 'gray',
                    line_width = 0.25,
                    fill_alpha = 1)


p.add_tools(HoverTool(renderers = [counties],                   # Sets what shows up when we hover
                     tooltips =[('County', '@NAME'),
                               ('State', '@STATEFP'),
                               ('Growth (%)', '@per_growth')]))
# Define color palettes
palette = brewer['OrRd'][5]                                     # Color Paletts
palette = palette[::-1]                 # reverse order of colors so higher values have darker colors      
    
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 100)
    
color_bar = ColorBar(color_mapper = color_mapper,           # Color bar, doesn't want to work, keep tyring
                    label_standoff = 8,
                    width = 500,
                    height = 20,
                    border_line_color = None,
                    location = (25,-120),
                    orientation = 'horizontal')    



show(p)

Two items of note:  
1. The above TWO cells must be run to create the plot. 
2. The map will auto open in a new tab 

References:
http://docs.bokeh.org/en/0.11.0/docs/gallery/choropleth.html  

https://towardsdatascience.com/walkthrough-mapping-basics-with-bokeh-and-geopandas-in-python-43f40aa5b7e9  

https://docs.bokeh.org/en/latest/docs/user_guide/plotting.html